# Init

In [39]:
# init
from importlib import reload
from pprint import pprint
import os
import sys
from pathlib import Path
import pandas as pd
def reloadPckg():
    reload(tgm)
    reload(too)
    reload(osmt)

sys.path.append('..')
sys.path.append('../tools')
devPath = "C:/Users/gonta/D/software developer/"
sys.path.append(os.path.join(devPath, "packages"))


import tests.osm_test as osmt
import toolsGeneral.main as tgm
import toolsOSM.overpass as too

In [70]:
# init
osm_meta = tgm.load(r"C:\Users\gonta\D\study\full stack\projects\administrative divisions query - levels scrape\OSM\get countries ids and add labels\data\osmMetaCountrDict.json")
len(osm_meta)

241

In [40]:
# init
import tools.logger_config as logconfig

# Cleaning

## import

In [1]:
dataDir = Path(os.path.join(os.getcwd(), '..', 'data', 'raw', 'osm countries queries'))
rawFiles = [f for f in dataDir.rglob('*.json')]

rawByCntr = {
    file.parent.name:tgm.load(str(file))['elements']
    for file in rawFiles
}
print(len(rawByCntr))

241

76


In [152]:
pprint({k:len(v) for k,v in rawByCntr.items()})

{'Albania': 390,
 'Algeria': 2148,
 'Andorra': 1,
 'Australia': 616,
 'Austria': 2178,
 'Austria--Liechtenstein': 1,
 'Austria-Switzerland': 1,
 'Bangladesh': 453,
 'Belarus': 1431,
 'Belgium': 4351,
 'Bhutan': 1191,
 'Brazil': 792,
 'Bulgaria': 113,
 'Cambodia': 316,
 'Chile': 420,
 'Colombia': 3806,
 'Czechia': 6281,
 'Denmark': 6,
 'Ecuador': 1240,
 'Estonia': 1,
 'Eswatini': 58,
 'FaroeIslands': 1,
 'Finland': 20,
 'France-Andorra': 1,
 'France-Belgium': 1,
 'France-Germany': 1,
 'France-Italy': 1,
 'France-Luxembourg': 1,
 'France-Monaco': 1,
 'France-Spain': 1,
 'France-Switzerland': 1,
 'Georgia': 163,
 'Germany': 11264,
 'Germany--Switzerland': 1,
 'Germany-Austria': 1,
 'Greece': 589,
 'Guinea': 90,
 'Guinea-bissau': 47,
 'Hungary': 3179,
 'Ireland': 1,
 'IsleOfMan': 1,
 'Jordan': 106,
 'Kyrgyzstan': 68,
 'Laos': 170,
 'Latvia': 1,
 'Lebanon': 36,
 'Libya': 24,
 'Lithuania': 4997,
 'Mauritania': 26,
 'Mexico': 3129,
 'Moldova': 38,
 'Mongolia': 363,
 'Montenegro': 1,
 'Myanmar

## use only sovereign countries

In [2]:
sovereign_countries = tgm.load(os.path.join('..', 'data', 'sovereign countries.json'))
len(sovereign_countries)

214

In [4]:
df_by_cntr = {k:elems for k,elems in rawByCntr.items() if k in sovereign_countries}
len(df_by_cntr)

64

## normalize

In [5]:
df_by_cntr = {k:too.normalizeOSM(elems) for k,elems in df_by_cntr.items()}
len(df_by_cntr)

64

In [6]:
for k,df in df_by_cntr.items():
    cntr = df[df['tags.admin_level']=='2']
    print([k, cntr['tags.name'].iloc[0]])

['Albania', 'Shqipëria']
['Algeria', 'Algérie ⵍⵣⵣⴰⵢⴻⵔ الجزائر']
['Andorra', 'Andorra']
['Australia', 'Australia']
['Austria', 'Österreich']
['Bangladesh', 'বাংলাদেশ']
['Belarus', 'Беларусь']
['Belgium', 'België / Belgique / Belgien']
['Bhutan', 'འབྲུག་ཡུལ།']
['Brazil', 'Brasil']
['Bulgaria', 'България']
['Cambodia', 'ព្រះរាជាណាចក្រ\u200bកម្ពុជា']
['Chile', 'Chile']
['Colombia', 'Colombia']
['Czechia', 'Česko']
['Denmark', 'Danmark']
['Ecuador', 'Ecuador']
['Estonia', 'Eesti']
['Eswatini', 'Eswatini']
['FaroeIslands', 'Føroyar']
['Finland', 'Suomi / Finland']
['Georgia', 'საქართველო']
['Germany', 'Deutschland']
['Greece', 'Ελλάς']
['Guinea', 'Guinée']
['Guinea-bissau', 'Guiné-Bissau']
['Hungary', 'Magyarország']
['Ireland', 'Éire / Ireland']
['IsleOfMan', 'Isle of Man']
['Jordan', 'الأردن']
['Kyrgyzstan', 'Кыргызстан']
['Laos', 'ປະເທດລາວ']
['Latvia', 'Latvija']
['Lebanon', 'لبنان']
['Libya', 'ليبيا']
['Lithuania', 'Lietuva']
['Mauritania', 'Mauritanie موريتانيا']
['Mexico', 'México']
['

In [7]:
# stores data by country
raw_normalized_dir = os.path.join('../data/normalized')
for k,df in df_by_cntr.items():
    path = os.path.join(raw_normalized_dir, k, 'rawNormalized.pkl')
    tgm.dump(path, df)

# Test elems

In [58]:
# import
dataDir = Path(os.path.join(os.getcwd(), '..', 'data/normalized'))
rawFiles = [f for f in dataDir.rglob('*.pkl')]
print(len(rawFiles))

df_by_cntr = {
    file.parent.name:tgm.load(str(file))
    for file in rawFiles
}
print(len(df_by_cntr))

64
64


## 1. basic elems test

In [10]:
def test(cntr, df):
    print(cntr)
    return testbasic.osm_basic_test(df)

osm_basic_test = {cntr: test(cntr, df) for cntr, df in df_by_cntr.items()}

Albania
 * missing names:  0
 * relations from other countries:  0
Algeria
 * missing names:  0
 * relations from other countries:  0
Andorra
 * missing names:  0
 * relations from other countries:  0
Australia
 * missing names:  0
 * relations from other countries:  0
Austria
 * missing names:  0
 * relations from other countries:  1
Bangladesh
 * missing names:  1
 * relations from other countries:  9
Belarus
 * missing names:  0
 * relations from other countries:  6
Belgium
 * missing names:  0
 * relations from other countries:  7
Bhutan
 * missing names:  0
 * relations from other countries:  0
Brazil
 * missing names:  0
 * relations from other countries:  28
Bulgaria
 * missing names:  0
 * relations from other countries:  0
Cambodia
 * missing names:  0
 * relations from other countries:  0
Chile
 * missing names:  0
 * relations from other countries:  1
Colombia
 * missing names:  8
 * relations from other countries:  0
Czechia
 * missing names:  0
 * relations from other coun

In [11]:
tgm.dump(os.path.join('..', 'data', 'tests results', 'osm_basic_test.pkl'), osm_basic_test)

## 2. osm duplicates test

In [66]:
reloadPckg()

## load processed data and re-run tests

In [55]:
odt_processed_info = tgm.load(os.path.join(os.getcwd(), "../data/tests results/osm duplicates test", "odt_processed_info.pkl"))
len(odt_processed_info)

1341

In [56]:
odt_processed_info = {k:v for k,v in odt_processed_info.items() if not k[1]=='19415099'}
len(odt_processed_info)

1339

In [67]:
dup_test_logger = logconfig.initiate_logger('../data/tests results/osm duplicates test/odt_dup_test.log', 'dup_test_logger')

odt_test_res = osmt.countries_run_test(
    df_by_cntr, 
    osmt.osm_duplicates_test_center,
    dup_test_logger,
    save=True,
    save_dir=os.path.join(os.getcwd(), f"../data/tests results/osm duplicates test"),
    processed_info = odt_processed_info
)

[INFO] : [64/64] Processing Vietnam
[INFO] :  - duplicate elements by id: 13
[INFO] :  - testing osm center: 13
[INFO] :   * ['19415099', '1844412']:


 - Getting center of child: 
 - Getting parent that contains center: 


[INFO] :   * finished: status: ('ok', ''); result: True
[INFO] :   * ['19415099', '1902967']:


 - Getting center of child: 
 - Getting parent that contains center: 


[INFO] :   * finished: status: ('ok', ''); result: False
[INFO] :   * ['12138923', '19539163']:
[INFO] :   * already tested: pass
[INFO] :   * ['13470001', '19539163']:
[INFO] :   * already tested: pass
[INFO] :   * ['12138923', '19539228']:
[INFO] :   * already tested: pass
[INFO] :   * ['13470001', '19539159']:
[INFO] :   * already tested: pass
[INFO] :   * ['13470001', '19539161']:
[INFO] :   * already tested: pass
[INFO] :   * ['12138923', '19539173']:
[INFO] :   * already tested: pass
[INFO] :   * ['12138923', '19539197']:
[INFO] :   * already tested: pass
[INFO] :   * ['12138923', '19539198']:
[INFO] :   * already tested: pass
[INFO] :   * ['13470001', '19539198']:
[INFO] :   * already tested: pass
[INFO] :   * ['18305258', '18439792']:


 - Getting center of child: 


[INFO] :   * finished: status: ('error', 'missing_center'); result: 
[INFO] :   * ['18305258', '19377767']:


 - Getting center of child: 


[INFO] :   * finished: status: ('error', 'missing_center'); result: 


In [68]:
odt_test_res_not_empty = {
    k:v 
    for k,v in odt_test_res.items() 
    if v['test_res']
    and v['duplicate.keep_elems']
    and v['duplicate.delete_elems']
}
len(odt_test_res_not_empty)

1

In [69]:
odt_test_res_not_empty

{'Vietnam': {'test_res': {('19415099', '1844412'): {'status': 'ok',
    'result': True,
    'data': {'version': 0.6,
     'generator': 'Overpass API 0.7.62.8 e802775f',
     'osm3s': {'timestamp_osm_base': '2025-10-26T17:49:55Z',
      'timestamp_areas_base': '2025-10-26T11:58:06Z',
      'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
     'elements': [{'type': 'relation', 'id': 1844412}]}},
   ('19415099', '1902967'): {'status': 'ok',
    'result': False,
    'data': {'version': 0.6,
     'generator': 'Overpass API 0.7.62.8 e802775f',
     'osm3s': {'timestamp_osm_base': '2025-10-26T17:49:55Z',
      'timestamp_areas_base': '2025-10-26T11:58:06Z',
      'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
     'elements': []}},
   ('18305258', '18439792'): {'status': 'error',
    'error_type': 'missing_center',
    'data': {'version': 0.6,
   

## merge and save results

In [43]:
test_res_to_merge = odt_test_res_not_empty
print(len(test_res_to_merge))

1


In [ ]:
for k,v in test_res_to_merge.items():
    data_path = os.path.join(os.getcwd(), f"../data/tests results/osm duplicates test/{k}/{k}_test_res.pkl")
    if not os.path.exists(data_path):
        tgm.dump(data_path, v)
    else:
        test_res_old_value = tgm.load(data_path)
        test_res_to_save= {}
        test_res_to_save['test_res'] = test_res_old_value['test_res'] | test_res_to_merge[k]['test_res']
        test_res_to_save['duplicate.keep_elems'] = list(set(test_res_old_value['duplicate.keep_elems']) | set(test_res_to_merge[k]['duplicate.keep_elems']))
        test_res_to_save['duplicate.delete_elems'] = list(set(test_res_old_value['duplicate.delete_elems']) | set(test_res_to_merge[k]['duplicate.delete_elems']))
        tgm.dump(data_path, test_res_to_save)

In [53]:
data_dir = Path(os.path.join(os.getcwd(), '..', 'data/tests results/osm duplicates test'))
odt_test_res_files = [f for f in data_dir.rglob('*/*.pkl')]
print(len(odt_test_res_files))

odt_test_res_old = {
    file.parent.name:tgm.load(str(file))
    for file in odt_test_res_files
}

64


## compute new odt_processed_info

In [53]:
data_dir = Path(os.path.join(os.getcwd(), '..', 'data/tests results/osm duplicates test'))
odt_test_res_files = [f for f in data_dir.rglob('*/*.pkl')]
print(len(odt_test_res_files))

odt_processed_info_new = {}
for file in odt_test_res_files:
    test_res = tgm.load(str(file))['test_res']
    for k,v in test_res.items():
        if v['status'] == 'ok':
            odt_processed_info_new[(file.parent.name, k[0], k[1])] = dict(filter(lambda item: item[0] in ['status','result'], v.items()))

64


In [54]:
tgm.dump(os.path.join(os.getcwd(), "../data/tests results/osm duplicates test", "odt_processed_info.pkl"), odt_processed_info_new)

In [84]:
tests_res_not_empty = {
    k:v 
    for k,v in odt_test_res_old.items() 
    if v['test_res']
    or v['duplicate.keep_elems']
    or v['duplicate.delete_elems']
}
len(tests_res_not_empty)

25

In [97]:
temp = []
for k,v in tests_res_not_empty.items():
    # temp.append(v['test_res'])
    for ek,ev in v['test_res'].items():
        temp.append(ev['data'])

In [51]:
data_dir = Path(os.path.join(os.getcwd(), '..', 'data/tests results/osm duplicates test'))
odt_test_res_files = [f for f in data_dir.rglob('*/*.pkl')]
print(len(odt_test_res_files))

odt_test_res_old = {
    file.parent.name:tgm.load(str(file))
    for file in odt_test_res_files
}

64


In [52]:
temp_list = []
for ck,cv in odt_test_res_old.items():
    for ek,ev in cv['test_res'].items():
        temp_list.append(ev['data'].keys())
temp = pd.Series(temp_list)
temp.value_counts()


(version, generator, osm3s, elements)    1343
Name: count, dtype: int64

In [31]:
import copy
odt_test_res_new = copy.deepcopy(odt_test_res_old)
for ck,cv in odt_test_res_old.items():
    for ek,ev in odt_test_res_new[ck]['test_res'].items():
        ev['data'] = ev['data']['data']

In [40]:
temp_list = []
for ck,cv in odt_test_res_new.items():
    for ek,ev in cv['test_res'].items():
        temp_list.append(ev['data'].keys())
temp = pd.Series(temp_list)
temp.value_counts()

(version, generator, osm3s, elements)    1343
Name: count, dtype: int64

In [50]:
for k,v in list(odt_test_res_new.items()):
    tgm.dump(os.path.join(os.getcwd(),  f"../data/tests results/osm duplicates test/{k}/{k}_test_res.pkl"),v)